## Desafio Semana 8 (Enem)

In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    OneHotEncoder, Binarizer, KBinsDiscretizer,
    MinMaxScaler, StandardScaler, PolynomialFeatures
)

In [2]:
%matplotlib inline
from IPython.core.pylabtools import figsize


figsize(12, 12)

sns.set()

In [383]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [531]:
list(test.columns)

In [532]:
colunas_de_teste_para_treino = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
                             'NU_IDADE', 'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
                             'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
                             'TP_DEPENDENCIA_ADM_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ',
                             'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO',
                             'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'CO_PROVA_CN',
                             'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_CN',
                             'NU_NOTA_CH', 'NU_NOTA_LC','NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO',
                             'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
                             'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q006',
                             'Q024', 'Q025', 'Q026', 'Q027', 'Q047']

__Função para zerar as notas de quem faltou ou foi eliminado__ ___(Parametros == 0 e 2)___

In [535]:
def zerando_notas(df, coluna_presenca, coluna_nota):
      for i in df[coluna_presenca].iteritems():
        valor = (i[1])
       # print('valor', valor)
        if valor == 0 or valor == 2:
            df.loc[i[0], coluna_nota] = 0.0


__Agora sim fazendo o treino__


In [544]:
#colunas selecionadas: so as notas msm
#['NU_INSCRICAO', 'NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_MT','NU_NOTA_LC', 'NU_NOTA_REDACAO']

In [562]:
train.shape

(13730, 167)

In [569]:
df_train = train[['NU_INSCRICAO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC','NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_MT','NU_NOTA_LC', 'NU_NOTA_REDACAO']]

In [570]:
#preparando a base de treino

#nota de CN
zerando_notas(df_train, 'TP_PRESENCA_CN', 'NU_NOTA_CN')
#nota de CH
zerando_notas(df_train, 'TP_PRESENCA_CH', 'NU_NOTA_CH')
#nota de LC
zerando_notas(df_train, 'TP_PRESENCA_LC', 'NU_NOTA_LC')
#nota de REDACAO
zerando_notas(df_train, 'TP_PRESENCA_LC', 'NU_NOTA_REDACAO')
#nota de MT
zerando_notas(df_train, 'TP_PRESENCA_LC', 'NU_NOTA_MT')

In [571]:
df_train.shape

(13730, 9)

In [572]:
#variaveis para treino
x_treino = df_train.drop(columns = ['NU_INSCRICAO','NU_NOTA_MT'])
y_treino = df_train['NU_NOTA_MT']

In [573]:
variaveis_test = ['NU_INSCRICAO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']

In [574]:
#preparando a base de test
#chamando todas as funcoes usadas na base de treino pq o test tem o mesm comportamento
df_test = test[variaveis_test]

In [575]:
#nota de CN
zerando_notas(df_test, 'TP_PRESENCA_CN', 'NU_NOTA_CN')
#nota de CH
zerando_notas(df_test, 'TP_PRESENCA_CH', 'NU_NOTA_CH')
#nota de LC
zerando_notas(df_test, 'TP_PRESENCA_LC', 'NU_NOTA_LC')
#nota de REDACAO
zerando_notas(df_test, 'TP_PRESENCA_LC', 'NU_NOTA_REDACAO')

In [576]:
df_test.shape

(4576, 8)

In [577]:
test.shape

(4576, 47)

In [578]:
x_test = df_test.drop(columns = 'NU_INSCRICAO')

In [579]:
#agora sim aplicando a regressao linear

lm = LinearRegression()
lm.fit(x_treino, y_treino)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [580]:
pred = lm.predict(x_test)

In [581]:
lm.intercept_.round(2)

-0.74

In [582]:
#Gerando a tabela com os resultados
tabela_resultado = df_test[['NU_INSCRICAO']]
tabela_resultado['NU_NOTA_MT'] = np.around(pred,2)

#exportando o .csv
tabela_resultado.to_csv('answer.csv', index=False, header=True)

C:\Users\mxapi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



__Conclusão so fri um pouco na resolução desse desafio porquê da ultima vez eu tirei os registros que continham Nan.__

__O resultado desse desafio foi 93% e uns quebrados... acho q se por mais parametros pode melhorar o resultado.__